### NYC Taxi - analysis
##### Rafael Takata García

---
The purpose of this repo is to get to know the framework of `MLFlow`

To instance the mlflow UI run this in GIT:  uv run mlflow ui --backend-store-uri sqlite:///mlflow.db
Libraries

In [12]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
import mlflow

Read the data and creation of some variables:

In [13]:
def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

Separate the data in train and validation

In [14]:
categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

Test a model:

In [15]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/16 20:38:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:38:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:38:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run finalizado. RMSE = 8.9926


Model Registry Example:

In [16]:
import pickle

from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

New experiment

In [17]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")
mlflow.sklearn.autolog()

In [18]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [19]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            #!mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:39:25 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
2025/10/16 20:39:53 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
c:\Users\Rafa\apps\5to semestre\CdD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/16 20:40:11 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.


Loggin models from the code.

In [20]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:40:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:40:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:40:57 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/16 20:41:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:41:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-model'.


##### Otra manera de registrar entradas

In [11]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/14 21:49:23 WARNING mlflow.tracking._model_registry.fluent: Run with id 56e8da6d66a14b3fb4f0d080daf70dc6 has no artifacts at artifact path 'model', registering model based on models:/m-d9e8827421cd45f4a412107f1a3212ef instead
Created version '4' of model 'nyc-taxi-model'.


A través de MLFLOWCLient

In [22]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [23]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

Asignar alias

In [59]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=7
)

In [32]:
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1760499488151, current_stage='None', deployment_job_state=None, description='', last_updated_timestamp=1760499488151, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='e985e9a3996045e0a8dcf8e768b0d843', run_link='', source=('file:c:/Users/Rafa/apps/5to '
 'semestre/CdD/nyc-taxi-predictions-2025/notebooks/mlruns/2/models/m-339228a3eb254d7c880aac8ea25a82e1/artifacts'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [30]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [31]:
client.update_model_version(
    name="nyc-taxi-model",
    version=3,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=[], creation_timestamp=1760499938793, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1760669255035, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='de9e06489bbf49a0a494a2063b6d2929', run_link=None, source='models:/m-a6e9a5ba0bb5488b9220a1c670447e6e', status='READY', status_message=None, tags={}, user_id=None, version=3>

Obteniendo modelos del registro de modelos

In [33]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([ 4.98661726, 24.1397534 , 26.9647779 , ..., 19.72875784,
       15.33514682, 22.07257903], shape=(44218,))

In [45]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([ 4.98661726, 24.1397534 , 26.9647779 , ..., 19.72875784,
       15.33514682, 22.07257903], shape=(44218,))

Probamos los modelos en nuevos datos

In [35]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [37]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [39]:
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\Users\\Rafa\\apps\\5to semestre\\CdD\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [40]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [41]:
X_test = preprocess(df, dv)

In [42]:
target = "duration"
y_test = df[target].values

In [60]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

{'rmse': 6.01147524270201}

In [55]:
test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)

{'rmse': 6.01147524270201}